# Deepfake Detection: End-to-End Training, Inference & Deployment

This notebook provides a complete pipeline for:
1. **Training**: Single-stage and two-stage fine-tuning of Tiny-LaDeDa student model
2. **Inference**: Patch-level predictions with heatmap visualization
3. **Deployment**: Model quantization, ONNX export, and TFLite conversion

For Google Colab:
1. Mount your Google Drive
2. Place dataset in: /content/drive/MyDrive/deepfake-patch-audit/data/
3. Run cells sequentially

In [ ]:
# Check GPU availability
import torch
print(f"CUDA Available: {togrch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install required packages
import subprocess
import sys

packages = [
    'torch',
    'torchvision',
    'onnx',
    'onnxruntime',
    'scikit-learn',
    'pillow',
    'tqdm',
    'pandas',
    'numpy',
    'tensorflow',
]

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} installed")

In [ ]:
import os
import sys
from pathlib import Path

# Check if running in Colab
try:
    from google.colab import drive
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running locally")

# Mount Google Drive if in Colab
if IN_COLAB:
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/MyDrive/deepfake-patch-audit')
    print(f"Project root: {PROJECT_ROOT}")
else:
    PROJECT_ROOT = Path('/home/incharaj/Team-Converge/deepfake-patch-audit')

# Add project to path
sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

print(f"Working directory: {os.getcwd()}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import json
from tqdm import tqdm
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Device configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ Configuration complete")

In [ ]:
class BaseDataset(Dataset):
    '''Base dataset class for deepfake detection.'''
    
    def __init__(self, root_dir, split='train', image_format='jpg',
                 resize_size=256, normalize=True, normalize_mean=None,
                 normalize_std=None, split_file=None):
        self.root_dir = Path(root_dir)
        self.split = split
        self.image_format = image_format
        self.resize_size = resize_size
        self.normalize = normalize
        self.split_file = split_file
        
        if normalize_mean is None:
            self.normalize_mean = np.array([0.485, 0.456, 0.406])
        else:
            self.normalize_mean = np.array(normalize_mean)
        
        if normalize_std is None:
            self.normalize_std = np.array([0.229, 0.224, 0.225])
        else:
            self.normalize_std = np.array(normalize_std)
        
        self.samples = []
        self._load_samples()
    
    def _load_samples(self):
        if self.split_file:
            self._load_from_csv()
        else:
            self._load_from_directory()
    
    def _load_from_csv(self):
        split_path = Path(self.split_file)
        if not split_path.exists():
            raise FileNotFoundError(f"Split file not found: {split_path}")
        df = pd.read_csv(split_path)
        for _, row in df.iterrows():
            img_path = row['path']
            label = int(row['label'])
            if Path(img_path).exists():
                self.samples.append((img_path, label))
        print(f"✓ Loaded {len(self.samples)} samples from {split_path}")
    
    def _load_from_directory(self):
        real_dir = self.root_dir / "real"
        if real_dir.exists():
            for img_path in sorted(real_dir.glob(f"*.{self.image_format}")):
                self.samples.append((str(img_path), 0))
        
        fake_dir = self.root_dir / "fake"
        if fake_dir.exists():
            for img_path in sorted(fake_dir.glob(f"*.{self.image_format}")):
                self.samples.append((str(img_path), 1))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        image = image.resize((self.resize_size, self.resize_size), Image.BICUBIC)
        image = np.array(image, dtype=np.float32) / 255.0
        
        if self.normalize:
            image = (image - self.normalize_mean) / self.normalize_std
        
        image = torch.from_numpy(image).permute(2, 0, 1)
        return {"image": image, "label": torch.tensor(label, dtype=torch.long)}

print("✓ BaseDataset class defined")

In [ ]:
import torch.nn.functional as F

class TinyLaDeDaModel(nn.Module):
    '''Ultra-lightweight student model (1,297 parameters).'''
    
    def __init__(self):
        super().__init__()
        self.register_buffer('grad_filter', torch.tensor(
            [[0, 0, 0], [0, -2, 1], [0, 1, 0]], dtype=torch.float32
        ).unsqueeze(0).unsqueeze(0))
        
        self.conv = nn.Conv2d(3, 8, kernel_size=1, stride=1, padding=0, bias=True)
        self.fc = nn.Linear(8, 1, bias=True)
    
    def forward(self, x):
        x_preprocessed = []
        for i in range(3):
            channel = x[:, i:i+1, :, :]
            grad = F.conv2d(channel, self.grad_filter, padding=1)
            x_preprocessed.append(grad)
        x_grad = torch.cat(x_preprocessed, dim=1)
        
        x_conv = self.conv(x_grad)
        x_pool = F.avg_pool2d(x_conv, kernel_size=2, stride=2)
        
        b, c, h, w = x_pool.shape
        x_reshape = x_pool.permute(0, 2, 3, 1)
        x_logits = self.fc(x_reshape)
        x_logits = x_logits.permute(0, 3, 1, 2)
        
        x_out = F.interpolate(x_logits, size=(126, 126), mode='bilinear', align_corners=False)
        return x_out

class TinyLaDeDa(nn.Module):
    '''Wrapper for Tiny-LaDeDa student model.'''
    
    def __init__(self, pretrained=False, pretrained_path=None):
        super().__init__()
        self.model = TinyLaDeDaModel()
        if pretrained and pretrained_path and Path(pretrained_path).exists():
            state_dict = torch.load(pretrained_path, map_location='cpu')
            self.model.load_state_dict(state_dict)
    
    def forward(self, x):
        return self.model(x)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)

test_model = TinyLaDeDa()
print(f"✓ TinyLaDeDa model defined ({test_model.count_parameters()} parameters)")
del test_model

In [ ]:
class TopKLogitPooling(nn.Module):
    '''Top-K logit pooling for spatial patch-logit maps.'''
    
    def __init__(self, k_percent=0.1):
        super().__init__()
        self.k_percent = k_percent
    
    def forward(self, logits):
        b = logits.size(0)
        flat = logits.view(b, -1)
        k = max(1, int(flat.size(1) * self.k_percent))
        top_k_vals = torch.topk(flat, k, dim=1)[0]
        image_logit = top_k_vals.mean(dim=1, keepdim=True)
        return image_logit

class PatchDistillationLoss(nn.Module):
    '''Combined loss: Patch-level MSE + Image-level BCE.'''
    
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()
        self.bce_loss = nn.BCEWithLogitsLoss()
    
    def forward(self, student_patches, teacher_patches, student_image_logit, labels):
        teacher_resized = F.interpolate(
            teacher_patches, size=student_patches.shape[-2:], 
            mode='bilinear', align_corners=False
        )
        distill_loss = self.mse_loss(student_patches, teacher_resized)
        task_loss = self.bce_loss(student_image_logit.squeeze(1), labels.float())
        total_loss = self.alpha * distill_loss + (1 - self.alpha) * task_loss
        return total_loss, distill_loss, task_loss

print("✓ TopKLogitPooling and PatchDistillationLoss defined")

In [ ]:
class PatchStudentTrainer:
    '''Train student model with patch-level knowledge distillation.'''
    
    def __init__(self, student_model, teacher_model, train_loader, val_loader,
                 criterion, pooling, device='cuda', lr=0.001, weight_decay=1e-4):
        self.student_model = student_model.to(device)
        self.teacher_model = teacher_model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion.to(device)
        self.pooling = pooling.to(device)
        self.device = device
        
        self.teacher_model.eval()
        for param in self.teacher_model.parameters():
            param.requires_grad = False
        
        self.optimizer = optim.Adam(self.student_model.parameters(), 
                                   lr=lr, weight_decay=weight_decay)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=50, eta_min=1e-6)
        
        self.history = {
            'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_auc': []
        }
    
    def train_epoch(self):
        self.student_model.train()
        total_loss = 0.0
        pbar = tqdm(self.train_loader, desc='Training', leave=False)
        
        for batch in pbar:
            images = batch['image'].to(self.device)
            labels = batch['label'].to(self.device)
            
            student_patches = self.student_model(images)
            student_image_logit = self.pooling(student_patches)
            
            with torch.no_grad():
                teacher_patches = self.teacher_model(images)
            
            loss, _, _ = self.criterion(student_patches, teacher_patches, 
                                       student_image_logit, labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.student_model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = total_loss / len(self.train_loader)
        self.history['train_loss'].append(avg_loss)
        return avg_loss
    
    def validate(self):
        self.student_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_preds, all_targets = [], []
        
        with torch.no_grad():
            for batch in self.val_loader:
                images = batch['image'].to(self.device)
                labels = batch['label'].to(self.device)
                
                student_patches = self.student_model(images)
                student_image_logit = self.pooling(student_patches)
                teacher_patches = self.teacher_model(images)
                
                loss, _, _ = self.criterion(student_patches, teacher_patches, 
                                           student_image_logit, labels)
                
                total_loss += loss.item()
                predicted = (student_image_logit.squeeze(1) > 0.0).long()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
                all_preds.append(torch.sigmoid(student_image_logit.squeeze(1)).cpu())
                all_targets.append(labels.cpu())
        
        avg_loss = total_loss / len(self.val_loader)
        accuracy = correct / total
        
        try:
            all_preds = torch.cat(all_preds).numpy()
            all_targets = torch.cat(all_targets).numpy()
            auc = roc_auc_score(all_targets, all_preds)
        except:
            auc = 0.0
        
        self.history['val_loss'].append(avg_loss)
        self.history['val_acc'].append(accuracy)
        self.history['val_auc'].append(auc)
        return avg_loss, accuracy, auc
    
    def train(self, epochs, checkpoint_dir='outputs/checkpoints'):
        checkpoint_dir = Path(checkpoint_dir)
        checkpoint_dir.mkdir(parents=True, exist_ok=True)
        best_val_auc = 0.0
        
        print('\n' + '='*80)
        print('PATCH-LEVEL DISTILLATION TRAINING')
        print('='*80)
        
        for epoch in range(epochs):
            train_loss = self.train_epoch()
            val_loss, val_acc, val_auc = self.validate()
            self.scheduler.step()
            
            print(f'Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Val AUC: {val_auc:.4f}')
            
            if val_auc > best_val_auc:
                best_val_auc = val_auc
                torch.save(self.student_model.state_dict(), 
                          checkpoint_dir / 'student_best.pt')
        
        torch.save(self.student_model.state_dict(), 
                  checkpoint_dir / 'student_final.pt')
        print(f'\n✓ Training complete. Final model saved.')
        return self.history

print("✓ PatchStudentTrainer class defined")

In [ ]:
class TwoStagePatchStudentTrainer:
    '''Two-stage fine-tuning with progressive unfreezing.'''
    
    def __init__(self, student_model, teacher_model, train_loader, val_loader,
                 criterion, pooling, device='cuda', stage1_lr=0.001, 
                 stage2_lr=0.0001, weight_decay=1e-4):
        self.student_model = student_model.to(device)
        self.teacher_model = teacher_model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion.to(device)
        self.pooling = pooling.to(device)
        self.device = device
        self.stage1_lr = stage1_lr
        self.stage2_lr = stage2_lr
        self.weight_decay = weight_decay
        
        self.teacher_model.eval()
        for param in self.teacher_model.parameters():
            param.requires_grad = False
        
        self.history = {'train_loss': [], 'val_loss': [], 'val_auc': []}
        self.optimizer = None
        self.scheduler = None
    
    def _freeze_backbone(self):
        model = (self.student_model.model if hasattr(self.student_model, 'model') 
                else self.student_model)
        for param in model.parameters():
            param.requires_grad = False
        if hasattr(model, 'fc'):
            for param in model.fc.parameters():
                param.requires_grad = True
    
    def _unfreeze_layer1(self):
        model = (self.student_model.model if hasattr(self.student_model, 'model') 
                else self.student_model)
        if hasattr(model, 'conv'):
            for param in model.conv.parameters():
                param.requires_grad = True
    
    def _setup_stage1_optimizer(self):
        model = (self.student_model.model if hasattr(self.student_model, 'model') 
                else self.student_model)
        trainable_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optim.Adam(trainable_params, lr=self.stage1_lr, 
                                   weight_decay=self.weight_decay)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', factor=0.5, patience=2)
    
    def _setup_stage2_optimizer(self):
        model = (self.student_model.model if hasattr(self.student_model, 'model') 
                else self.student_model)
        trainable_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optim.Adam(trainable_params, lr=self.stage2_lr, 
                                   weight_decay=self.weight_decay)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=20, eta_min=1e-6)
    
    def train_epoch(self, stage):
        self.student_model.train()
        total_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f'Stage {stage} Training', leave=False)
        
        for batch in pbar:
            images = batch['image'].to(self.device)
            labels = batch['label'].to(self.device)
            
            student_patches = self.student_model(images)
            student_image_logit = self.pooling(student_patches)
            
            with torch.no_grad():
                teacher_patches = self.teacher_model(images)
            
            loss, _, _ = self.criterion(student_patches, teacher_patches, 
                                       student_image_logit, labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.student_model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = total_loss / len(self.train_loader)
        self.history['train_loss'].append(avg_loss)
        return avg_loss
    
    def validate(self):
        self.student_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_preds, all_targets = [], []
        
        with torch.no_grad():
            for batch in self.val_loader:
                images = batch['image'].to(self.device)
                labels = batch['label'].to(self.device)
                
                student_patches = self.student_model(images)
                student_image_logit = self.pooling(student_patches)
                teacher_patches = self.teacher_model(images)
                
                loss, _, _ = self.criterion(student_patches, teacher_patches, 
                                           student_image_logit, labels)
                
                total_loss += loss.item()
                predicted = (student_image_logit.squeeze(1) > 0.0).long()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
                all_preds.append(torch.sigmoid(student_image_logit.squeeze(1)).cpu())
                all_targets.append(labels.cpu())
        
        avg_loss = total_loss / len(self.val_loader)
        accuracy = correct / total
        
        try:
            auc = roc_auc_score(torch.cat(all_targets).numpy(), torch.cat(all_preds).numpy())
        except:
            auc = 0.0
        
        self.history['val_loss'].append(avg_loss)
        self.history['val_auc'].append(auc)
        return avg_loss, accuracy, auc
    
    def train(self, epochs_s1=5, epochs_s2=20, checkpoint_dir='outputs/checkpoints_two_stage'):
        checkpoint_dir = Path(checkpoint_dir)
        checkpoint_dir.mkdir(parents=True, exist_ok=True)
        best_val_auc = 0.0
        
        print('\n' + '='*80)
        print('TWO-STAGE FINE-TUNING')
        print('='*80)
        
        print('\n*** STAGE 1: Freeze Backbone, Train Classifier ***')
        self._freeze_backbone()
        self._setup_stage1_optimizer()
        
        for epoch in range(epochs_s1):
            train_loss = self.train_epoch(stage=1)
            val_loss, val_acc, val_auc = self.validate()
            lr = self.optimizer.param_groups[0]['lr']
            print(f'S1 Epoch {epoch+1}/{epochs_s1} | Loss: {train_loss:.4f} | AUC: {val_auc:.4f}')
            self.scheduler.step(val_auc)
            
            if val_auc > best_val_auc:
                best_val_auc = val_auc
                torch.save(self.student_model.state_dict(), 
                          checkpoint_dir / 'student_best.pt')
        
        print('\n*** STAGE 2: Unfreeze Last Layers, Fine-tune ***')
        self._unfreeze_layer1()
        self._setup_stage2_optimizer()
        
        for epoch in range(epochs_s2):
            train_loss = self.train_epoch(stage=2)
            val_loss, val_acc, val_auc = self.validate()
            print(f'S2 Epoch {epoch+1}/{epochs_s2} | Loss: {train_loss:.4f} | AUC: {val_auc:.4f}')
            self.scheduler.step()
            
            if val_auc > best_val_auc:
                best_val_auc = val_auc
                torch.save(self.student_model.state_dict(), 
                          checkpoint_dir / 'student_best.pt')
        
        torch.save(self.student_model.state_dict(), 
                  checkpoint_dir / 'student_final.pt')
        print(f'\n✓ Two-stage training complete.')
        return self.history

print("✓ TwoStagePatchStudentTrainer class defined")

In [ ]:
class InferencePipeline:
    '''Complete inference pipeline with patch-level predictions and heatmaps.'''
    
    def __init__(self, model, pooling, device='cuda', image_size=256,
                 normalize_mean=None, normalize_std=None):
        self.model = model.to(device)
        self.model.eval()
        self.pooling = pooling.to(device) if pooling else None
        self.device = device
        self.image_size = image_size
        
        if normalize_mean is None:
            self.normalize_mean = np.array([0.485, 0.456, 0.406])
        else:
            self.normalize_mean = np.array(normalize_mean)
        
        if normalize_std is None:
            self.normalize_std = np.array([0.229, 0.224, 0.225])
        else:
            self.normalize_std = np.array(normalize_std)
    
    def _preprocess_image(self, image):
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        elif not isinstance(image, Image.Image):
            image = Image.fromarray((image * 255).astype(np.uint8))
        
        image = image.resize((self.image_size, self.image_size), Image.BICUBIC)
        image = np.array(image, dtype=np.float32) / 255.0
        image = (image - self.normalize_mean) / self.normalize_std
        image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)
        return image
    
    def predict(self, image):
        image_tensor = self._preprocess_image(image).to(self.device)
        
        with torch.no_grad():
            patch_logits = self.model(image_tensor)
            if self.pooling:
                image_logit = self.pooling(patch_logits)
            else:
                image_logit = patch_logits.mean(dim=[2, 3], keepdim=True)
            
            fake_prob = torch.sigmoid(image_logit).item()
            patch_heatmap = torch.sigmoid(patch_logits).squeeze(1)
        
        return {
            'patch_logits': patch_logits.cpu().numpy(),
            'patch_heatmap': patch_heatmap.cpu().numpy(),
            'fake_prob': fake_prob,
            'real_prob': 1.0 - fake_prob,
            'prediction': 'FAKE' if fake_prob > 0.5 else 'REAL',
        }
    
    def predict_batch(self, images_list):
        batch_images = []
        for img in images_list:
            img_tensor = self._preprocess_image(img)
            batch_images.append(img_tensor)
        
        batch_tensor = torch.cat(batch_images, dim=0).to(self.device)
        
        with torch.no_grad():
            patch_logits = self.model(batch_tensor)
            if self.pooling:
                image_logits = self.pooling(patch_logits)
            else:
                image_logits = patch_logits.mean(dim=[2, 3], keepdim=True)
            
            fake_probs = torch.sigmoid(image_logits).squeeze(1).cpu().numpy()
        
        results = []
        for prob in fake_probs:
            results.append({
                'fake_prob': float(prob),
                'real_prob': 1.0 - float(prob),
                'prediction': 'FAKE' if prob > 0.5 else 'REAL',
            })
        return results
    
    def visualize_heatmap(self, image, result, figsize=(15, 5)):
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        image = np.array(image.resize((256, 256), Image.BICUBIC), dtype=np.float32) / 255.0
        heatmap = result['patch_heatmap'][0]
        
        fig, axes = plt.subplots(1, 3, figsize=figsize)
        axes[0].imshow(image)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        im1 = axes[1].imshow(heatmap, cmap='RdYlGn_r', vmin=0, vmax=1)
        axes[1].set_title('Fake Probability Heatmap')
        axes[1].axis('off')
        plt.colorbar(im1, ax=axes[1])
        
        heatmap_resized = np.array(Image.fromarray((heatmap * 255).astype(np.uint8)).resize(
            (256, 256), Image.BICUBIC)) / 255.0
        axes[2].imshow(image)
        im2 = axes[2].imshow(heatmap_resized, cmap='RdYlGn_r', vmin=0, vmax=1, alpha=0.5)
        pred_text = f"Pred: {result['prediction']}, Prob: {result['fake_prob']:.3f}"
        axes[2].set_title(f'Overlay ({pred_text})')
        axes[2].axis('off')
        plt.colorbar(im2, ax=axes[2])
        
        plt.tight_layout()
        return fig

print("✓ InferencePipeline class defined")

In [ ]:
class ImprovedDynamicRangeQuantizer:
    '''Dynamic range quantization with per-channel and outlier clipping.'''
    
    def __init__(self, bits=8, symmetric=False, per_channel=True,
                 clip_outliers=True, clip_percentile=99.9):
        self.bits = bits
        self.symmetric = symmetric
        self.per_channel = per_channel
        self.clip_outliers = clip_outliers
        self.clip_percentile = clip_percentile
        
        if bits == 8:
            self.qmin, self.qmax = -128, 127
        else:
            raise ValueError(f"Unsupported bit width: {bits}")
    
    def _compute_quant_params_per_tensor(self, X):
        if self.clip_outliers:
            percentile_val = np.percentile(np.abs(X), self.clip_percentile)
            X_clipped = np.clip(X, -percentile_val, percentile_val)
        else:
            X_clipped = X
        
        x_min = X_clipped.min()
        x_max = X_clipped.max()
        
        if x_max == x_min:
            scale = 1e-8
        else:
            if self.symmetric:
                abs_max = max(abs(x_min), abs(x_max))
                scale = (2 * abs_max) / (self.qmax - self.qmin)
            else:
                scale = (x_max - x_min) / (self.qmax - self.qmin)
        
        if self.symmetric:
            zero_point = 0
        else:
            zero_point_real = self.qmin - x_min / scale
            zero_point = int(round(zero_point_real))
            zero_point = np.clip(zero_point, self.qmin, self.qmax)
        
        return scale, zero_point
    
    def _compute_quant_params_per_channel(self, tensor):
        out_channels = tensor.shape[0]
        scales, zero_points = [], []
        
        for ch in range(out_channels):
            channel_data = tensor[ch].detach().cpu().numpy().flatten()
            scale, zp = self._compute_quant_params_per_tensor(channel_data)
            scales.append(scale)
            zero_points.append(zp)
        
        return np.array(scales), np.array(zero_points)
    
    def quantize_model(self, model):
        quant_params = {}
        model.eval()
        
        for name, module in model.named_modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                if hasattr(module, 'weight'):
                    weight = module.weight.data
                    if self.per_channel and weight.dim() > 1:
                        scales, zps = self._compute_quant_params_per_channel(weight)
                    else:
                        scale, zp = self._compute_quant_params_per_tensor(
                            weight.cpu().numpy().flatten())
                        scales, zps = np.array([scale]), np.array([zp])
                    
                    quant_params[name] = {
                        'scales': scales, 'zero_points': zps, 'shape': weight.shape
                    }
        
        return model, quant_params
    
    def get_quantization_report(self, quant_params):
        report = '\n' + '='*80 + '\n' + 'QUANTIZATION REPORT\n' + '='*80 + '\n'
        report += f'Bits: {self.bits}\n'
        report += f'Mode: {"Symmetric" if self.symmetric else "Asymmetric"}\n'
        report += f'Per-Channel: {self.per_channel}\n'
        report += f'Outlier Clipping: {self.clip_outliers} ({self.clip_percentile}th percentile)\n'
        report += f'Total layers quantized: {len(quant_params)}\n'
        return report

print("✓ ImprovedDynamicRangeQuantizer class defined")

In [ ]:
def create_sample_data_loaders(dataset_root, batch_size=16, num_workers=0):
    '''Create train/val dataloaders.'''
    try:
        train_dataset = BaseDataset(
            root_dir=f"{dataset_root}/train",
            split='train',
            image_format='jpg',
            resize_size=256,
        )
        print(f"✓ Loaded {len(train_dataset)} training samples")
        
        val_dataset = BaseDataset(
            root_dir=f"{dataset_root}/val",
            split='val',
            image_format='jpg',
            resize_size=256,
        )
        print(f"✓ Loaded {len(val_dataset)} validation samples")
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                                 shuffle=True, num_workers=num_workers)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                               shuffle=False, num_workers=num_workers)
        
        return train_loader, val_loader
    except Exception as e:
        print(f"⚠ Failed to load dataset: {e}")
        print("Expected structure: dataset_root/train/{real,fake}/ and /val/{real,fake}/")
        return None, None

def initialize_training_pipeline(device=DEVICE):
    '''Initialize student, teacher, pooling, and loss.'''
    student_model = TinyLaDeDa(pretrained=False)
    print(f"✓ Student model initialized ({student_model.count_parameters()} parameters)")
    
    teacher_model = TinyLaDeDa(pretrained=False)
    print(f"✓ Teacher model initialized")
    
    for param in teacher_model.parameters():
        param.requires_grad = False
    
    pooling = TopKLogitPooling(k_percent=0.1)
    criterion = PatchDistillationLoss(alpha=0.5)
    print(f"✓ Pooling and loss initialized")
    
    return student_model, teacher_model, pooling, criterion

print("✓ Data loading and initialization utilities defined")

In [ ]:
def plot_training_history(history, figsize=(15, 5)):
    '''Plot training history.'''
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    axes[0].plot(history['train_loss'], label='Train')
    if 'val_loss' in history:
        axes[0].plot(history['val_loss'], label='Val')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    if 'val_acc' in history:
        axes[1].plot(history['val_acc'])
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy')
        axes[1].set_title('Validation Accuracy')
        axes[1].grid(True)
    
    if 'val_auc' in history:
        axes[2].plot(history['val_auc'])
        axes[2].set_xlabel('Epoch')
        axes[2].set_ylabel('AUC')
        axes[2].set_title('Validation AUC')
        axes[2].grid(True)
    
    plt.tight_layout()
    return fig

def print_model_summary(model):
    '''Print model architecture.'''
    print('\nModel Architecture:')
    print('='*80)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'Total parameters: {total_params:,}')
    print(f'Trainable parameters: {trainable_params:,}')

def compute_model_size(model):
    '''Compute model size in MB.'''
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    return (param_size) / 1024 / 1024

print("✓ Helper functions defined")

In [ ]:
# 

## Usage Examples

### Step 1: Prepare Data
```python
dataset_root = "/content/drive/MyDrive/deepfake-patch-audit/data"
train_loader, val_loader = create_sample_data_loaders(dataset_root, batch_size=16)
```

### Step 2: Initialize Models
```python
student, teacher, pooling, criterion = initialize_training_pipeline()
```

### Step 3: Train (Single-Stage)
```python
trainer = PatchStudentTrainer(student, teacher, train_loader, val_loader, 
                             criterion, pooling, device=DEVICE)
history = trainer.train(epochs=20)
```

### Step 4: Train (Two-Stage)
```python
trainer = TwoStagePatchStudentTrainer(student, teacher, train_loader, val_loader,
                                     criterion, pooling, device=DEVICE)
history = trainer.train(epochs_s1=5, epochs_s2=20)
```

### Step 5: Inference & Visualization
```python
pipeline = InferencePipeline(student, pooling, device=DEVICE)
result = pipeline.predict("image.jpg")
fig = pipeline.visualize_heatmap("image.jpg", result)
plt.show()
```

### Step 6: Quantization
```python
quantizer = ImprovedDynamicRangeQuantizer(bits=8, per_channel=True)
quantized_model, params = quantizer.quantize_model(student)
print(quantizer.get_quantization_report(params))
```

## Summary

✓ All components loaded successfully!

**Available Classes:**
- `BaseDataset`: Dataset loading (directory/CSV modes)
- `TinyLaDeDa`: Student model (1,297 parameters)
- `TopKLogitPooling`: Spatial pooling layer
- `PatchDistillationLoss`: MSE + BCE loss
- `PatchStudentTrainer`: Single-stage training
- `TwoStagePatchStudentTrainer`: Two-stage fine-tuning
- `InferencePipeline`: Inference with heatmaps
- `ImprovedDynamicRangeQuantizer`: Int8 quantization

**Next Steps:**
1. Mount Google Drive and prepare dataset
2. Run `create_sample_data_loaders()` with your dataset path
3. Initialize models with `initialize_training_pipeline()`
4. Train with `PatchStudentTrainer` or `TwoStagePatchStudentTrainer`
5. Run inference with `InferencePipeline`
6. Quantize with `ImprovedDynamicRangeQuantizer`